In [1]:
import sys
import os
import base64
import time
import wave
import sda
import torch
import librosa
import soundfile as sf
import IPython

import houndify
from ipywebrtc import VideoStream, AudioRecorder, CameraStream, AudioStream

In [2]:
sys.path.insert(1, '../speech-driven-animation/')

In [3]:
camera = CameraStream(constraints=
                      {'facing_mode': 'user',
                       'audio': True,
                       'video': { 'width': 640, 'height': 480 }
                       })
camera

CameraStream(constraints={'facing_mode': 'user', 'audio': True, 'video': {'width': 640, 'height': 480}})

In [4]:
recorder = AudioRecorder(stream=camera)
recorder

AudioRecorder(audio=Audio(value=b'', format='webm'), stream=CameraStream(constraints={'facing_mode': 'user', '…

In [11]:
# save output
recorder.save("output.wav")  # will save to test.mp3

# convert to proper wav file
x,_ = librosa.load('output.wav', sr=16000)
sf.write('tmp.wav', x, 16000)
wave.open('tmp.wav','r')

CLIENT_ID = "Rco-d2ARwyAUSvklEH_ePg=="
CLIENT_KEY = "V6TzSgIOkx0CW4AkBSxgLNrcHMEZ26PSB4uLaG-n8t6PsjZSEzBF-aGfXQv-mWXha0_grVB-TO5MGdKhx9XNFA=="
AUDIO_FILE = "tmp.wav"

BUFFER_SIZE = 256

audio = wave.open(AUDIO_FILE)
if audio.getsampwidth() != 2:
  print("%s: wrong sample width (must be 16-bit)" % fname)
  sys.exit()
if audio.getframerate() != 8000 and audio.getframerate() != 16000:
  print("%s: unsupported sampling frequency (must be either 8 or 16 khz)" % fname)
  sys.exit()
if audio.getnchannels() != 1:
  print("%s: must be single channel (mono)" % fname)
  sys.exit()


audio_size = audio.getnframes() * audio.getsampwidth()
audio_duration = audio.getnframes() / audio.getframerate()
chunk_duration = BUFFER_SIZE * audio_duration / audio_size


#
# Simplest HoundListener; just print out what we receive.
# You can use these callbacks to interact with your UI.
#
class MyListener(houndify.HoundListener):
  def onPartialTranscript(self, transcript):
    print("Partial transcript: " + transcript)
  def onFinalResponse(self, response):
    #print("Final response: " + str(response))
    
    response_bytes = response["AllResults"][0]["ResponseAudioBytes"]
#     os.system("say {}".format(response_text)) 
#     print("Response bytes:" + response_bytes)
    print("Final response: " + str(response["AllResults"][0]["SpokenResponse"]))

    # convert bytes string to binary
    decoded = base64.decodebytes(response_bytes.encode("ascii"))

    # writes response to wav file
    with open('myfile.wav', mode='wb') as f:
      f.write(decoded)
    
#     va = sda.VideoAnimator(gpu=0, model_path="timit")# Instantiate the animator
#     vid, aud = va("IMG_9997.bmp", "myfile.wav")
#     va.save_video(vid, aud, "generated_sabrina.mp4")
      
    
  def onError(self, err):
    print("Error: " + str(err))


client = houndify.StreamingHoundClient(CLIENT_ID, CLIENT_KEY, "test_user", enableVAD=False)
client.setLocation(37.388309, -121.973968)
client.setSampleRate(audio.getframerate())
client.setHoundRequestInfo("ResponseAudioVoice", "Sharon")
client.setHoundRequestInfo("ResponseAudioShortOrLong", "Short")

client.start(MyListener())

while True:
  chunk_start = time.time()

  samples = audio.readframes(BUFFER_SIZE)
  if len(samples) == 0: break
  if client.fill(samples): break

result = client.finish() # returns either final response or error
IPython.display.Audio("myfile.wav")

ValueError: No data, did you record anything?

In [31]:
# open video
video2 = VideoStream.from_url('generated_sabrina.mp4')
video2

NameError: name 'video2' is not defined